# Тестовое задание для стажеров аналитиков-технологов

Необходимо разработать некоторые части приложения для учета книг в библиотеке. Описание данных, с которыми будет работать приложение – ниже.

Приложение для учета книг в библиотеке должно:

1. Хранить названия книг, ФИО авторов, наименования издательств, год издания.
2. Учитывать имеющиеся в библиотеке экземпляры конкретной книги.
3. Учитывать студентов, которым выдавалась конкретная книга. При каждой выдаче книги студенту, фиксируется дата выдачи. При возврате – дата возврата книги.

## Задания:

1. Опишите модель данных (в любом удобном для вас представлении) для обслуживания библиотеки. Это может быть описание таблиц с типами данных, диаграмма – что угодно.

Указанную модель данных удобно представить в виде следующей ER-диаграммы:

![ER-диаграмма БД](diag.png)

2. Напишите SQL-запрос, который бы возвращал самого популярного автора за год. Запрос должен основываться на модели данных, которую вы описали в задании 1. 

Запишем указанный запрос и, для наглядности, выполним его средствами Python:

In [59]:
import psycopg2 as pg

class DBexecutor:
    
    def __init__(self, params):
        self.params = params
        self.conn, self.curs = self.connect_db()
        
    def connect_db(self):
        try:
            conn = pg.connect(**self.params)
            conn.set_session(autocommit=True)
            if conn:
                curs = conn.cursor()
                return conn, curs
        except Exception:
            print('Ошибка подключения к БД')
            return None, None
        
    def __del__(self):
        if self.conn:
            self.curs.close()
            self.conn.close()

In [60]:
params = {
    'dbname': 'library',
    'user': 'homutovan',
    'password': ''
        }

db = DBexecutor(params)

df = pd.read_sql("""
                select first_name, middle_name, last_name
                from books_students bs
                inner join books b on book_id = b.id
                inner join author a2 on author_id = a2.id
                inner join first_name fn on first_name_id = fn.id
                inner join middle_name mn on middle_name_id = mn.id
                inner join last_name ln2 on last_name_id = ln2.id
                where bs.date_of_issue between '2019-01-01' and '2019-12-31'
                group by first_name, middle_name, last_name 
                order by count(bs.id) desc
                limit 1;
                """, db.conn)

df

,first_name,middle_name,last_name
0,Вячеслав,Иванович,Букур


In [61]:
del db

3. Определите понятие «злостный читатель».  Предложите алгоритм для поиска самого злостного читателя библиотеки. На любом языке программирования опишите алгоритм поиска такого читателя. Алгоритм должен основываться на модели данных, которую вы описали в задании 1.

Определим понятие «злостный читатель» - пусть «злостным» является такой читатель, который с начала года допускал задержку возврата книг, в среднем, более чем на 15 дней. Максимально допустимый срок возврата книг примем равным 30 дням. Выполним запрос для поиска таких читателей в БД:

In [215]:
import pandas as pd

db = DBexecutor(params)

df = pd.read_sql("""
                select first_name, middle_name, last_name, faculty 
                from books_students bs 
                inner join student s on student_id = s.id
                inner join first_name fn on first_name_id = fn.id
                inner join middle_name mn on middle_name_id = mn.id
                inner join last_name ln2 on last_name_id = ln2.id
                inner join faculty f on faculty_id = f.id
                where bs.date_of_issue between '2020-01-01' and '2020-06-26'
                group by first_name, middle_name, last_name, faculty 
                having sum(date_of_return - date_of_issue - 30) / count(bs.id) > 15
                order by last_name;
                """, db.conn)
df

,first_name,middle_name,last_name,faculty
0,Адольф,Нестеровна,Абрамович,Факультет автоматики и вычислительной техники ...
1,Виталий,Эльевич,Агафонов,Факультет энергетики (ФЭН)
2,Эммануил,Самойлович,Бондарев,Механико -технологический факультет (МТФ)
3,Татьяна,Соломоновна,Вершинин,Факультет летательных аппаратов (ФЛА)
4,Александра,Павлович,Горб,Факультет мехатроники и автоматизации (ФМА)
5,Станислав,Борисовна,Козлов,Механико -технологический факультет (МТФ)
6,Феоктист,Макарович,Куприна,Факультет радиотехники и электроники (РЭФ)
7,Рашель,Федотович,Ляпин,Факультет летательных аппаратов (ФЛА)
8,Рашель,Арсеньевич,Мельников-Печерский,Факультет прикладной математики и информатики ...
9,Анатолий,Константиновна,Назаренко,Факультет летательных аппаратов (ФЛА)


In [63]:
del db

В с помощью данного запроса мы объединяем таблицы books_students, student, first_name, middle_name, last_name и faculty, группируем записи по ФИО студентов, для каждой полученной группы выполняем расчет отношения числа дней просрочки к числу записей о получении и возврате книги и фильтруем полученный результат, отсекая значения менее 15.

Аналогичного результата мы можем достичь запросив данные от БД, выполнив дальнейшую обработку средствами Python:

In [154]:
import datetime

db = DBexecutor(params)

db.curs.execute("""
                select * 
                from books_students bs 
                inner join student s on student_id = s.id
                inner join first_name fn on first_name_id = fn.id
                inner join middle_name mn on middle_name_id = mn.id
                inner join last_name ln2 on last_name_id = ln2.id
                inner join faculty f on faculty_id = f.id
                """)

data = db.curs.fetchall()
df = pd.DataFrame(data)
data = df[[3, 4, 11, 13, 15, 17]]
data.columns = ['issue', 'return', 'first_name', 'middle_name', 'last_name', 'faculty']
filter_after = data['issue'] >= datetime.date(2020, 1, 1)
filter_before = data['issue'] <= datetime.date(2020, 6, 26)
data = data.loc[filter_before & filter_after]

In [219]:
data['delta'] = data['return'] - data['issue'] - pd.Timedelta(30, unit='d')
group_data = data.groupby(['first_name', 'middle_name', 'last_name', 'faculty']).agg(['sum', 'count']).reset_index()
group_data['av_delay'] = group_data[['delta'][0]]['sum'] / group_data[['delta'][0]]['count']
filter_delay = group_data['av_delay'].dt.round(freq = 'D') > pd.Timedelta(15, unit='d')
group_data.loc[filter_delay].sort_values('last_name').reset_index()[['first_name', 'middle_name', 'last_name', 'faculty']]

,first_name,middle_name,last_name,faculty
,,,,
0,Адольф,Нестеровна,Абрамович,Факультет автоматики и вычислительной техники ...
1,Виталий,Эльевич,Агафонов,Факультет энергетики (ФЭН)
2,Эммануил,Самойлович,Бондарев,Механико -технологический факультет (МТФ)
3,Татьяна,Соломоновна,Вершинин,Факультет летательных аппаратов (ФЛА)
4,Александра,Павлович,Горб,Факультет мехатроники и автоматизации (ФМА)
5,Станислав,Борисовна,Козлов,Механико -технологический факультет (МТФ)
6,Феоктист,Макарович,Куприна,Факультет радиотехники и электроники (РЭФ)
7,Рашель,Федотович,Ляпин,Факультет летательных аппаратов (ФЛА)
8,Рашель,Арсеньевич,Мельников-Печерский,Факультет прикладной математики и информатики ...
